In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../')

import skeletor as sk

import tqdm

import networkx as nx

from anytree import Node, RenderTree

In [ ]:
data = '../Data/cube.npy'
dsFactor = 1
neighborDistance = 25
np.random.seed(10)

#%matplotlib qt
with open(data, 'rb') as f:
    rawData = np.load(f)
    
order = np.arange(rawData.shape[0])
np.random.shuffle(order)
points = rawData[order][::dsFactor,:]

#plt.scatter(scatterPoints[:,0], scatterPoints[:,2], s=.2)
#plt.show()

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(points[:,0], points[:,1], points[:,2], s=1)
plt.show()

In [ ]:
octree = sk.Octree(points, 1000, 1/128, True, neighborMethod='discrete')

In [ ]:
fig = octree.plot()
fig.set_size_inches((9,9))
plt.show()

In [ ]:
plt.hist(octree.vDimArr.flatten())
plt.show()

In [ ]:
%matplotlib inline
centroids = octree.getCentroids()
adjMat = octree.getAdjMat()

fig = plt.figure(figsize=(10, 15))
ax1 = fig.add_subplot(1, 2, 1, projection='3d')

ax1.scatter(points[:,0], points[:,1], points[:,2], s=5, alpha=.1)

ax2 = fig.add_subplot(1, 2, 2, projection='3d')

ax2.scatter(centroids[:,0], centroids[:,1], centroids[:,2], s=1)

for i in range(len(adjMat)):
    edgeIndices = np.where(adjMat[i] > 0)[0]
    for j in range(len(edgeIndices)):
        ax2.plot(*list(zip(centroids[i], centroids[edgeIndices[j]])))

fig.tight_layout()
#plt.savefig('cube_discrete_skeleton.png')
plt.show()

In [ ]:
dominantDirections = []
adjMat = octree.getAdjMat()
centroids = octree.getCentroids()

for i in tqdm.tqdm(range(len(centroids))):
    hist, thetaBins, phiBins = sk.angularHistogramAroundPoint(centroids, i, adjMat, smoothing=21)
    peakDirections = sk.findDominantHistogramDirections(hist, thetaBins, phiBins, normalize=True)

    dominantDirections.append(peakDirections)

In [ ]:
fig = plt.figure(figsize=(10, 15))
ax = fig.add_subplot(projection='3d')

for i in range(len(centroids)):
    for j in range(len(dominantDirections[i])):
        linePoints = np.array([centroids[i], centroids[i] - .1*dominantDirections[i][j]])
        ax.plot(linePoints[:,0], linePoints[:,1], linePoints[:,2], linewidth=3, c='tab:blue')

plt.show()

In [ ]:
%matplotlib inline

skelPoints, skelAdjMat = octree.skeletonize(threshold=-.7, closed=False, mergeThreshold=1, secondRoundNeighborDetection=True, allowMergePoints=True)

centroids = octree.getCentroids()
adjMat = octree.getAdjMat()

fig = plt.figure(figsize=(15, 10))
ax1 = fig.add_subplot(1, 3, 1, projection='3d')

ax1.scatter(points[:,0], points[:,1], points[:,2], s=5, alpha=.1)

ax2 = fig.add_subplot(1, 3, 2, projection='3d')

ax2.scatter(centroids[:,0], centroids[:,1], centroids[:,2], s=1)

for i in range(len(adjMat)):
    edgeIndices = np.where(adjMat[i] > 0)[0]
    for j in range(len(edgeIndices)):
        ax2.plot(*list(zip(centroids[i], centroids[edgeIndices[j]])))

ax3 = fig.add_subplot(1, 3, 3, projection='3d')

ax3.scatter(skelPoints[:,0], skelPoints[:,1], skelPoints[:,2], s=2, alpha=1)
for i in range(len(skelAdjMat)):
    edgeIndices = np.where(skelAdjMat[i] > 0)[0]
    for j in range(len(edgeIndices)):
        ax3.plot(*list(zip(skelPoints[i], skelPoints[edgeIndices[j]])))

ax1.set_title('Point Cloud')
ax2.set_title('Reduced Representation')
ax3.set_title('Skeleton')
        
fig.tight_layout()
#plt.savefig('cube_skeletonization.png', bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(10, 20))
nBoxesArr = [100, 500, 1000, 2000, 5000]

for k in tqdm.tqdm(range(len(nBoxesArr))):

    octree = sk.Octree(points, nBoxesArr[k], 1/128, neighborMethod='discrete')
    
    skelPoints, skelAdjMat = octree.skeletonize(threshold=-.7, closed=True, mergeThreshold=1, secondRoundNeighborDetection=True, allowMergePoints=True)

    centroids = octree.getCentroids()
    adjMat = octree.getAdjMat()


    ax1 = fig.add_subplot(len(nBoxesArr), 2, 2*k+1, projection='3d')
    
    ax1.scatter(points[:,0], points[:,1], points[:,2], s=5, alpha=.01)
    ax1.scatter(centroids[:,0], centroids[:,1], centroids[:,2], s=2)

    for i in range(len(adjMat)):
        edgeIndices = np.where(adjMat[i] > 0)[0]
        for j in range(len(edgeIndices)):
            ax1.plot(*list(zip(centroids[i], centroids[edgeIndices[j]])))

    ax2 = fig.add_subplot(len(nBoxesArr), 2, 2*k+2, projection='3d')

    ax2.scatter(skelPoints[:,0], skelPoints[:,1], skelPoints[:,2], s=2, alpha=1)
    for i in range(len(skelAdjMat)):
        edgeIndices = np.where(skelAdjMat[i] > 0)[0]
        for j in range(len(edgeIndices)):
            ax2.plot(*list(zip(skelPoints[i], skelPoints[edgeIndices[j]])))

    ax1.set_title(f'Reduced Representation (N = {nBoxesArr[k]})')
    ax2.set_title(f'Skeleton (N = {nBoxesArr[k]})')

fig.tight_layout()
#plt.savefig('cube_skeletonization.png', bbox_inches='tight')
plt.show()